In [1]:
%%time
import pandas as pd
import numpy as np
import torch
import os
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup, BertForSequenceClassification

from collections import defaultdict
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

RANDOM_SEED = 42


CPU times: user 721 ms, sys: 60 ms, total: 781 ms
Wall time: 806 ms


In [2]:
'''
column_names = ['content', 'label', 'hotel_name']
df = pd.DataFrame(columns = column_names)

root_dir = "./Crawler/CrawlerToBooking/"
directories = ['negativeReviews', 'positiveReviews']

for label, directory in enumerate(directories):
    txt_path = os.path.join(root_dir, directory)
    for filename in os.listdir(txt_path):
        hotel_name = os.path.splitext(filename)[0]
        with open(os.path.join(txt_path, filename)) as f:
            content = f.read()
            dict_new = {"content":content.strip().replace("\n",""), "label":str(label), "hotel_name":hotel_name}
            df = df.append(dict_new, ignore_index=True)
df.to_csv("reviews.csv", index=False)
'''

'\ncolumn_names = [\'content\', \'label\', \'hotel_name\']\ndf = pd.DataFrame(columns = column_names)\n\nroot_dir = "./Crawler/CrawlerToBooking/"\ndirectories = [\'negativeReviews\', \'positiveReviews\']\n\nfor label, directory in enumerate(directories):\n    txt_path = os.path.join(root_dir, directory)\n    for filename in os.listdir(txt_path):\n        hotel_name = os.path.splitext(filename)[0]\n        with open(os.path.join(txt_path, filename)) as f:\n            content = f.read()\n            dict_new = {"content":content.strip().replace("\n",""), "label":str(label), "hotel_name":hotel_name}\n            df = df.append(dict_new, ignore_index=True)\ndf.to_csv("reviews.csv", index=False)\n'

In [3]:
df = pd.read_csv('reviews.csv')
df.sample(10)

,content,label,hotel_name
64160,早餐人太多排隊排很久,0,娜路彎大酒店329
19452,無,0,花蓮鳥窩青年旅舍3
266740,很舒適的旅館和服務，下次會再來...早餐好吃極了...,1,東驛商旅 西寧館11
126829,早餐有點差,0,雙星大飯店1508
129917,地點有點尷尬，周遭的華陰街晚上基本上都大門深鎖沒有宵夜，因此不如南陽街；也由於鄰近店家很早就...,0,龍蝦先生的秘密巢穴設計青旅7
122265,沒有窗戶,0,台北大亞帝國H709
276984,房間乾淨舒適，cp值高！值得推薦！,1,新京站民宿63
91502,房間打掃不完整 桌上迎賓糖果裡居然有已經吃過的糖果紙包裝,0,昭盛52行館18
240231,床很舒服 床間也很棒行李部的服務員也很好 很樂於助人,1,義大皇家酒店55
258390,早餐還有吐司、貝果，水果切盤，飲料咖啡無限喝，床很舒適，吧檯員工新來的很風趣、很健談、重點會...,1,Formosa 1018


In [4]:
PRE_TRAINED_MODEL_NAME = 'bert-base-chinese'

In [5]:
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [6]:
# sample_txt = df.content[150]

In [7]:
'''
tokens = tokenizer.tokenize(sample_txt)
token_ids = tokenizer.convert_tokens_to_ids(tokens)

print(f' Sentence: {sample_txt}')
print(f'   Tokens: {tokens}')
print(f'Token IDs: {token_ids}')
'''

"\ntokens = tokenizer.tokenize(sample_txt)\ntoken_ids = tokenizer.convert_tokens_to_ids(tokens)\n\nprint(f' Sentence: {sample_txt}')\nprint(f'   Tokens: {tokens}')\nprint(f'Token IDs: {token_ids}')\n"

In [8]:
'''
encoding = tokenizer.encode_plus(
  sample_txt,
  max_length=512,
  truncation=True,
  add_special_tokens=True, 
  return_token_type_ids=False,
  pad_to_max_length=True,
  return_attention_mask=True,
  return_tensors='pt',  
)

encoding.keys()
'''

"\nencoding = tokenizer.encode_plus(\n  sample_txt,\n  max_length=512,\n  truncation=True,\n  add_special_tokens=True, \n  return_token_type_ids=False,\n  pad_to_max_length=True,\n  return_attention_mask=True,\n  return_tensors='pt',  \n)\n\nencoding.keys()\n"

In [9]:
# print(len(encoding['input_ids'][0]))
# encoding['input_ids'][0]

In [10]:
# print(len(encoding['attention_mask'][0]))
# encoding['attention_mask']

In [11]:
# tokenizer.convert_ids_to_tokens(encoding['input_ids'][0])

In [12]:
MAX_LEN = 512

In [13]:
class GPReviewDataset(Dataset):

  def __init__(self, reviews, targets, tokenizer, max_len):
    self.reviews = reviews
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len
  
  def __len__(self):
    return len(self.reviews)
  
  def __getitem__(self, item):
    review = str(self.reviews[item])
    target = self.targets[item]

    encoding = self.tokenizer.encode_plus(
      review,
      add_special_tokens=True,
      truncation=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt',
    )

    return {
      'review_text': review,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'targets': torch.tensor(target, dtype=torch.long)
    }

In [14]:
df_train, df_test = train_test_split(df, test_size=0.1, random_state=RANDOM_SEED)
df_val, df_test = train_test_split(df_test, test_size=0.5, random_state=RANDOM_SEED)

In [15]:
df_train.label.sample(10)

76802     0
174405    1
273718    1
262133    1
116991    0
242545    1
131065    0
266830    1
334871    1
6681      0
Name: label, dtype: int64

In [16]:
def create_data_loader(df, tokenizer, max_len, batch_size):
  ds = GPReviewDataset(
    reviews=df.content.to_numpy(),
    targets=df.label.to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len
  )

  return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=4
  )

In [17]:
BATCH_SIZE = 16

train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)

In [18]:
class SentimentClassifier(nn.Module):

  def __init__(self, n_classes):
    super(SentimentClassifier, self).__init__()
    self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
  
  def forward(self, input_ids, attention_mask):
    _, pooled_output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask
    )
    output = self.drop(pooled_output)
    return self.out(output)

In [19]:
class_names = ['negative', 'positive']
model = SentimentClassifier(len(class_names))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
    model = nn.DataParallel(model)

model = model.to(device)



Let's use 2 GPUs!


In [20]:
EPOCHS = 4

optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss().to(device)

In [21]:
def train_epoch(
  model, 
  data_loader, 
  loss_fn, 
  optimizer, 
  device, 
  scheduler, 
  n_examples
):
  model = model.train()

  losses = []
  correct_predictions = 0
  
  for d in data_loader:
    input_ids = d["input_ids"].to(device)
    attention_mask = d["attention_mask"].to(device)
    targets = d["targets"].to(device)

    outputs = model(
      input_ids=input_ids,
      attention_mask=attention_mask
    )

    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, targets)

    correct_predictions += torch.sum(preds == targets)
    losses.append(loss.item())

    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()

  return correct_predictions.double() / n_examples, np.mean(losses)

In [22]:
def eval_model(model, data_loader, loss_fn, device, n_examples):
  model = model.eval()

  losses = []
  correct_predictions = 0

  with torch.no_grad():
    for d in data_loader:
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)

      loss = loss_fn(outputs, targets)

      correct_predictions += torch.sum(preds == targets)
      losses.append(loss.item())

  return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
%%time

history = defaultdict(list)
best_accuracy = 0

for epoch in range(EPOCHS):

  print(f'Epoch {epoch + 1}/{EPOCHS}')
  print('-' * 10)

  train_acc, train_loss = train_epoch(
    model,
    train_data_loader,    
    loss_fn, 
    optimizer, 
    device, 
    scheduler, 
    len(df_train)
  )

  print(f'Train loss {train_loss} accuracy {train_acc}')

  val_acc, val_loss = eval_model(
    model,
    val_data_loader,
    loss_fn, 
    device, 
    len(df_val)
  )

  print(f'Val   loss {val_loss} accuracy {val_acc}')
  print()

  history['train_acc'].append(train_acc)
  history['train_loss'].append(train_loss)
  history['val_acc'].append(val_acc)
  history['val_loss'].append(val_loss)

  if val_acc > best_accuracy:
    torch.save(model.state_dict(), 'best_model_state.bin')
    best_accuracy = val_acc

Epoch 1/4
----------
